In [1]:
import pandas as pd



In [2]:
import bisect

In [3]:
from PIL import Image
import numpy as np
import io

In [4]:
def load_parquet_dataset(filename):
    df = pd.read_parquet(filename)
    labels = np.asarray(df['label'])
    images = [ np.asarray(Image.open(io.BytesIO(x['bytes']))) for x in df['image']]
    scaled = np.asarray(images) / 255.0
    image_reshaped = scaled.reshape(*scaled.shape, 1)
    return image_reshaped, labels

In [6]:
import os
os.environ["KERAS_BACKEND"] = "jax"
import keras

In [7]:
from keras import datasets, layers, models
from keras import regularizers

In [8]:
model = models.Sequential([
    layers.Conv2D(28, (3,3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(56, (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(56, (3,3), activation='relu')
])

model.add(layers.Flatten())
model.add(layers.Dense(56, activation='relu', kernel_regularizer=regularizers.L2(0.05)))
model.add(layers.Dense(10))




C:\Users\dmitr\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
train_images, train_labels = load_parquet_dataset("train-00000-of-00001.parquet")

In [11]:
test_images, test_labels = load_parquet_dataset("test-00000-of-00001.parquet")

In [12]:
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.9405 - loss: 0.4216 - val_accuracy: 0.9806 - val_loss: 0.1269
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.9749 - loss: 0.1424 - val_accuracy: 0.9834 - val_loss: 0.1137
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.9792 - loss: 0.1203 - val_accuracy: 0.9840 - val_loss: 0.0943
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.9820 - loss: 0.1062 - val_accuracy: 0.9800 - val_loss: 0.1139
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.9840 - loss: 0.0968 - val_accuracy: 0.9771 - val_loss: 0.1131
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.9858 - loss: 0.0891 - val_accuracy: 0.9838 - val_loss: 0.1002
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.9861 - loss: 0.0845 - val_accuracy: 0.9853 - val_loss: 0.0827
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.9870 -

In [13]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")

313/313 - 1s - 4ms/step - accuracy: 0.9832 - loss: 0.0836
Test accuracy: 0.98


In [14]:
from keras import Model
extracted = Model(model.inputs, model.layers[-1].output)


In [131]:
features[1]

array([ -3.3260803,  -1.5349885,  10.2836075,  -7.375522 ,  -1.6870499,
       -10.818754 ,  -3.4503078,  -4.160905 ,  -4.074495 , -10.990408 ],
      dtype=float32)

In [ ]:
for i, pred in enumerate(features):
    j = np.argmax(pred)
    if test_labels[i] != j:
        print (pred, test_labels[i], j)

In [15]:
def normalized_predictions(images, labels):
    assert(len(images) == len(labels))
    predictions = model.predict(images)
    normalized = np.array([x / np.sqrt(np.dot(x,x)) for x in predictions])
    return normalized
    

In [16]:
norms_train = normalized_predictions(train_images, train_labels)
norms_test = normalized_predictions(test_images, test_labels)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [198]:
norms_test[0]

array([-0.37355432, -0.02566617, -0.2309672 , -0.14557321, -0.04970531,
       -0.30704433, -0.55195504,  0.55261594, -0.2789246 ,  0.02601011],
      dtype=float32)

In [25]:
def fimi_encode(normalized, labels, levels):
    discreditized = [ np.array([bisect.bisect_left(levels, x) for x in pred]) for pred in normalized]
    total = len(levels) * discreditized[0].shape[0]
    fimi_lines = [
        "# attributes: %d properties: O(0,1,2,3,4,5,6,7,8,9)" % total
    ]
    for i, item in enumerate(discreditized):
        fimi = ''
        idx = 0
        for attr in item:
            for j in range(0, attr):
                if idx == 0 and j == 0:
                    fimi += str(j+idx)
                else:
                    fimi += " %d" % (j+idx)
            idx += len(levels)
        fimi += " | " + str(labels[i])
        fimi_lines.append(fimi)
    return fimi_lines


In [26]:
levels = list(map(lambda x: 0.1*x, range(-9, 10)))

In [22]:
levels

[-0.9,
 -0.8,
 -0.7000000000000001,
 -0.6000000000000001,
 -0.5,
 -0.4,
 -0.30000000000000004,
 -0.2,
 -0.1,
 0.0,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9]

In [27]:
train_fimi = fimi_encode(norms_train, train_labels, levels)

In [28]:
test_fimi = fimi_encode(norms_test, test_labels, levels)

In [29]:
with open("train.fimi", "w") as f:
    f.write("\n".join(train_fimi))

In [30]:
with open("test.fimi", "w") as f:
    f.write("\n".join(test_fimi))